# Introduction 

Supongamos que ud. es parte de un equipo de respuesta a crisis, el cual que quiere identificar como han estado respondiendo los hospitales a las colisione de tránsito en la ciudad de New York.

<center>
<img src="new_york_crash.jpg" width="450"><br/>
</center>

Antes de comenzar debemos ejecutar el siguiente bloque para inicializar todos los módulos necesarios.

In [98]:
import math
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPolygon

import folium
from folium import Choropleth, Marker, GeoJson
from folium.plugins import HeatMap, MarkerCluster

También vamos a usar la función `embed_map()` para asegurar la correcta visualización de los mapas.

In [99]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

# Ejercicios

## 1) Visualice los datos de colisiones

Corra el código en la celda de abajo para cargar el **GeoDataFrame** `collisions` con información de los choques importante de vehiculos motorizados entre 2013-2018.

In [100]:
collisions = gpd.read_file("../input/NYPD_Motor_Vehicle_Collisions/NYPD_Motor_Vehicle_Collisions.shp")
collisions.head(3)

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET,CROSS STRE,OFF STREET,...,CONTRIBU_2,CONTRIBU_3,CONTRIBU_4,UNIQUE KEY,VEHICLE TY,VEHICLE _1,VEHICLE _2,VEHICLE _3,VEHICLE _4,geometry
0,07/30/2019,0:00,BRONX,10464,40.841100,-73.784960,"(40.8411, -73.78496)",None,None,121 PILOT STREET,...,Unspecified,None,None,4180045,Sedan,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,None,None,POINT (1043750.211 245785.815)
1,07/30/2019,0:10,QUEENS,11423,40.710827,-73.770660,"(40.710827, -73.77066)",JAMAICA AVENUE,188 STREET,None,...,None,None,None,4180007,Sedan,Sedan,None,None,None,POINT (1047831.185 198333.171)
2,07/30/2019,0:25,None,None,40.880318,-73.841286,"(40.880318, -73.841286)",BOSTON ROAD,None,None,...,None,None,None,4179575,Sedan,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (1028139.293 260041.178)


Use las columnas **LATITUDE** y **LONGITUDE** para crear un mapa interactiva que visualice la distribución de los choques. Que tipo de mapa parece el más efectivo para mostrar esta información?

In [101]:
# calculamos el promedio de latitud y longitud
collitions_coord = collisions[['LATITUDE', 'LONGITUDE']]
mean_lat, mean_long = collitions_coord.mean()
print(mean_lat,mean_long)

40.692923335848114 -73.8689606520383


In [102]:
# creamos el mapa de New York
map1 = folium.Map(location=[40.7, -74], zoom_start=11)
# creamos mapa de calor con la distribución de los accidentes
HeatMap(data=collitions_coord, radius=10).add_to(map1);

embed_map(map1, "map1.html")

### 2) Entendiendo la cobertura hospitalaria

Ejecute la celda siguiente para cargar la información de los hospitales en New York.

In [103]:
hospitals = gpd.read_file("../input/nyu_2451_34494/nyu_2451_34494.shp")
hospitals.head(3)

,id,name,address,zip,factype,facname,capacity,capname,bcode,xcoord,ycoord,latitude,longitude,geometry
0,317000001H1178,BRONX-LEBANON HOSPITAL CENTER - CONCOURSE DIVI...,1650 Grand Concourse,10457,3102,Hospital,415,Beds,36005,1008872.0,246596.0,40.843490,-73.911010,POINT (1008872.000 246596.000)
1,317000001H1164,BRONX-LEBANON HOSPITAL CENTER - FULTON DIVISION,1276 Fulton Ave,10456,3102,Hospital,164,Beds,36005,1011044.0,242204.0,40.831429,-73.903178,POINT (1011044.000 242204.000)
2,317000011H1175,CALVARY HOSPITAL INC,1740-70 Eastchester Rd,10461,3102,Hospital,225,Beds,36005,1027505.0,248287.0,40.848060,-73.843656,POINT (1027505.000 248287.000)


Use las columnas **latitude** y **longitude** de este **DataFrame** para visualizar la ubicación de los hospitales.

In [120]:
# creamos el mapa de New York
map2 = folium.Map(location=[40.7, -74], zoom_start=11)
# agrego marcadores con ubicación de los hospitales
for idx, row in hospitals.iterrows():
    Marker([row.latitude, row.longitude]).add_to(map2)

embed_map(map2, "map2.html")

### 3) Cuando estaba el hospital más cercano a más de 10kms?

Crear un **DataFrame** `outside_range` con todos los registros en `collisions` de accidentes que ocurrieron a más de 10kms del hospital más cercano.

Comprobar que tanto `hospitals` como `collisions` tiene **EPSG 2263** como **CRS** y verificar que este sistema de coordenadas usa metros como unidad de medida (https://epsg.io/2263).

In [130]:
print("CRS de hospitals: {}".format(hospitals.crs))
print("CRS de collisions: {}".format(collisions.crs))
# creamos un buffer con el area a 10kms de cada hospital;
coverage = hospitals.geometry.buffer(10000)
coverage.head(3)

CRS de hospitals: epsg:2263
CRS de collisions: epsg:2263


0    POLYGON ((1018872.000 246596.000, 1018823.847 ...
1    POLYGON ((1021044.000 242204.000, 1020995.847 ...
2    POLYGON ((1037505.000 248287.000, 1037456.847 ...
dtype: geometry

In [108]:
# comparando distancias con lista de hospitales (INEFICIENTE)
def not_safe(p, h):
    min_dist = h.geometry.distance(p).min()
    return min_dist > 10000

import time
start_time = time.time()
# creamos filtro de los registros que no estan en safe_zone
# outside1 = collisions.geometry.apply(lambda p: not_safe(p, hospitals.geometry))
print("--- %s seconds ---" % (time.time() - start_time))

--- 3.62396240234375e-05 seconds ---


In [131]:
## Con shapely.MultiPolygon & unary_union
import time
start_time = time.time()
# cramos un MultiPolygon con la union de las zonas de los hospitales
safe_zone = coverage.unary_union
# creamos filtro de los registros que no estan en safe_zone
outside_filter = collisions.geometry.apply(lambda p: not p.within(safe_zone))
print("--- %s seconds ---" % (time.time() - start_time))
outside_range = collisions.loc[pd.Series(outside_filter)]
print("Hubo {:d} accidentes lejos de los hospitales".format(len(outside_range)))

--- 13.488712787628174 seconds ---
Hubo 39595 accidentes lejos de los hospitales


The next code cell calculates the percentage of collisions that occurred more than 10 kilometers away from the closest hospital.

In [148]:
percentage = round(100*len(outside_range)/len(collisions), 2)
print("Porcentaje de accidentes a más de 10ks de un hospital: {}%".format(percentage))

Porcentaje de accidentes a más de 10ks de un hospital: 15.12%


### 4) Creando un recomendador

Cuando un accidente ocurre lejos de un centro de asistencia se vuelve vital que las personas heridas sean transladadas al hospital más cercano.

Con este objetivo en mente, se decide crear un recomendador que:
- acepte la ubicacion del accidente en coordenadas **EPSG 2263** como entrada,
- encuentre el hospital más cercano (calculando distancias en **EPSG 2263**) y 
- devuelva el nombre del hospital más cercano.

In [133]:
def best_hospital(p):
    # Your code here
    closest = hospitals.iloc[hospitals.geometry.distance(p).idxmin()]
    return closest['name']

# Test your function: this should suggest CALVARY HOSPITAL INC
print(best_hospital(outside_range.geometry.iloc[0]))

CALVARY HOSPITAL INC


### 5) Cual es el hospital que tiene la mayor demanda?

Considerando solo las colisiones a más de 10kms en el DataFrame `outside_range`, cual es el hospital más recomendado según el criterio de ser el más cercano?

La respuesta debe ser un string de Python que coincida exactamente con el nombre del hospital tal cual es devuelto por la función creada en el punto **4)**.

In [149]:
demands = outside_range.geometry.apply(lambda p: best_hospital(p))
highest_demand = demands.value_counts().idxmax()
print("El hospital con la mayor demanda es: {}".format(highest_demand))

El hospital con la mayor demanda es: JAMAICA HOSPITAL MEDICAL CENTER


### 6) Donde se debería contruir nuevos hospitales?

Ejecute el bloque siguiente (sin cambios) para visualizar zona de cobertura de los hospitales junto con la distribución de los accidentes que ocurren a mas de 10 kms del hospital más cercano.

In [135]:
# visualizamos el area de cobertura junto a los accidentes
map3 = folium.Map(location=[40.7, -74], zoom_start=11)

# graficamos el area de 10km en el mapa (pasar a EPSG:4326)
GeoJson(coverage.to_crs(epsg=4326)).add_to(map3)

# creamos mapa de calor con los accidentes fuera de covertura
HeatMap(data=outside_range[['LATITUDE', 'LONGITUDE']], radius=10).add_to(map3);
folium.LatLngPopup().add_to(map3)

embed_map(map3, "map3.html")


La función `folium.LatLngPopup()` permite que al hacer *click* en cualquier punto del mapa se muestre un *pop-up* 
con la ubicación en latitud y longitud.

Supongamos que la ciudad de Nueva York lo contrata a ud. como consultor para definir la ubicacion de 2 nuevos hospitales El objetivo que se plantean es identificar 2 ubicaciones que bajen el porcentaje de accidentes fuera del area "segura", calculado en el paso **(3)**, a menos del 10%.

Usando el mapa anterior, y sin tener en cuenta posibles leyes de construcción o edificios a ser eliminados para construir los hospitales, debe indicar dos ubicaciones que permitan a la ciudad lograr este objetivo. Para esto complete los valores de latitud y longitud en las variables indicadas abajo para el hospital 1 y 2 respectivamente.

Luego ejecute el código de la celta (sin más cambios) para ver el efecto de los nuevos hospitales sobre la cobertura de accidentes.

La respuesta va a ser considerada correcta si los 2 hospitales permiten bajar el porcentaje a menos de 10%.

In [150]:
# Your answer here: proposed location of hospital 1
lat_1, long_1 = 40.678, -73.86

# Your answer here: proposed location of hospital 2
lat_2, long_2 = 40.68, -73.76

style = {'fillColor': '#3388ff', 'color': '#3388ff', "weight": 2, "opacity": 0.4}
new_style = {'fillColor': '#ff4455', 'color': '#ff4455', "weight": 2, "opacity": 0.6}

# make the map
m = folium.Map(location=[40.7, -74], zoom_start=11)

# Do not modify the code below this line
try:
    new_df = pd.DataFrame(
        {'Latitude': [lat_1, lat_2],
         'Longitude': [long_1, long_2]})
    new_gdf = gpd.GeoDataFrame(new_df, geometry=gpd.points_from_xy(new_df.Longitude, new_df.Latitude))
    new_gdf.crs = "epsg:4326"
    new_gdf = new_gdf.to_crs(epsg=2263)
    # get new percentage
    new_coverage = new_gdf.geometry.buffer(10000)
    new_my_union = new_coverage.geometry.unary_union
    new_outside_range = outside_range.loc[~outside_range["geometry"].apply(lambda x: new_my_union.contains(x))]
    new_percentage = round(100*len(new_outside_range)/len(collisions), 2)
    print("(EXITO) El porcentaje de accidentes a mas de 10kms de un hospital es: {}%".format(new_percentage))
    folium.GeoJson(coverage.to_crs(epsg=4326),
                   style_function=lambda x: style).add_to(m)
    folium.GeoJson(new_coverage.to_crs(epsg=4326),
                   style_function=lambda x: new_style).add_to(m)
    for idx, row in new_gdf.iterrows():
        Marker([row['Latitude'], row['Longitude']]).add_to(m)
    HeatMap(data=new_outside_range[['LATITUDE', 'LONGITUDE']], radius=10).add_to(m)
    folium.LatLngPopup().add_to(m)
except:
    print("Hint: Fill in locations corresponding to two relatively warmer areas of the heatmap.")
    
embed_map(m, 'q_6.html')

(EXITO) El porcentaje de accidentes a mas de 10kms de un hospital es: 8.58%
